In [51]:
from parsePython import *
from code2vec import *
from UseData import *
import numpy as np

In [73]:
def move(l1, l2):
    result = []
    flag = False
    for i in l2:
        for j in l1:
            if i==j:
                result.append(j)
                flag = True
                break
        if (flag == False):
            result.append(0)
        else:
            flag = False
    for k in l1:
        if k not in result:
            result.append(k)
    return result

def pad(x, size):
    if size <= len(x):
        return x[:size]
    out = [0]*size
    out[:len(x)] = x
    return out
    
def get_vec(java_data, py_data):
    _c2v = Code2Vec()
    _ud = UseData()
    _m, _a, _t = _ud.load_use_data()
    Xs = []
    Xt = []
    for i in java_data:
        py_token = cv.getToken(py_data[i])
        py_api = cv.getAPISequence(py_data[i])
        py_meth = cv.getMethodName(py_data[i])
        py_token_vec = _c2v.convert_tokens(py_token)
        py_apiseq_vec = _c2v.convert_apiseq(py_api)
        py_meth_vec= _c2v.convert_methname(py_meth)
        
        java_token_vec = _t[i-1]
        java_apiseq_vec = _a[i-1]
        java_meth_vec = _m[i-1]
        
        py_token_vec = move(py_token_vec, java_token_vec)
        py_apiseq_vec = move(py_apiseq_vec, java_apiseq_vec)
        py_meth_vec = move(py_meth_vec, java_meth_vec)
        
        padded_java_token_vec = pad(java_token_vec, 50)
        padded_java_apiseq_vec = pad(java_apiseq_vec, 30)
        padded_java_meth_vec = pad(java_meth_vec, 6)
        
        padded_py_token_vec = pad(py_token_vec, 50)
        padded_py_apiseq_vec = pad(py_apiseq_vec, 30)
        padded_py_meth_vec = pad(py_meth_vec, 6)
        
        temp = np.concatenate((padded_java_meth_vec, padded_java_apiseq_vec, padded_java_token_vec), axis=0)
        Xs.append(temp)
        temp = np.concatenate((padded_py_meth_vec, padded_py_apiseq_vec, padded_py_token_vec), axis=0)
        Xt.append(temp)
    return Xs, Xt

In [60]:
cv = CodeVisitor()
c2v = Code2Vec()
ud = UseData()
m, a, t = ud.load_use_data()

In [71]:
java_data = {17:"public static String getNextTag ( String xmlData , int position ) { String nextTag = null ; if ( xmlData != null && ! xmlData . isEmpty ( ) && position < xmlData . length ( ) && xmlData . substring ( position ) . contains ( "<" ) ) { while ( xmlData . charAt ( position ) != '<' ) { position ++ ; } int startIndex = position ; if ( xmlData . substring ( position ) . contains ( ">" ) ) { while ( xmlData . charAt ( position ) != '>' ) { position ++ ; } nextTag = xmlData . substring ( startIndex , position + 1 ) ; } } return nextTag ; } ",
7:"public float bottom ( float margin ) { return pageSize . bottom ( marginBottom + margin ) ; } ",
12:"public RGBColor toGreyScale ( ) { return new RGBColor ( 0.30 * getRed ( ) + 0.59 * getGreen ( ) + 0.11 * getBlue ( ) ) ; } ",
21:"public void addClientSocketListener ( ClientSocketListener listener ) { if ( listener == null ) throw new NullPointerException ( ) ; listeners . add ( listener ) ; }",
23:"final void addProcessToGcListLocked ( ProcessRecord proc ) { boolean added = false ; for ( int i = mProcessesToGc . size ( ) - 1 ; i >= 0 ; i -- ) { if ( mProcessesToGc . get ( i ) . lastRequestedGc < proc . lastRequestedGc ) { added = true ; mProcessesToGc . add ( i + 1 , proc ) ; break ; } } if ( ! added ) { mProcessesToGc . add ( 0 , proc ) ; } } ",
34:"public boolean isFocused ( ) { if ( m_Control == null ) return false ; return m_Control . isFocusControl ( ) ; } ",
36:"public static long getFreeDiskSpace ( boolean checkInternal ) { String status = Environment . getExternalStorageState ( ) ; long freeSpace = 0 ; if ( status . equals ( Environment . MEDIA_MOUNTED ) ) { freeSpace = freeSpaceCalculation ( Environment . getExternalStorageDirectory ( ) . getPath ( ) ) ; } else if ( checkInternal ) { freeSpace = freeSpaceCalculation ( \"/\" ) ; } else { return - 1 ; } return freeSpace ; }",
38:"public void removePropertyChangeListener ( String name , PropertyChangeListener pcl ) { m_bcSupport . removePropertyChangeListener ( name , pcl ) ; } ",
40:"public void incDfsUsed ( long value ) { used . addAndGet ( value ) ; } ",
49:"public void removeElements ( List elements ) { if ( elements . size ( ) > 0 ) { fElements . removeAll ( elements ) ; if ( fTable != null ) { fTable . remove ( elements . toArray ( ) ) ; } dialogFieldChanged ( ) ; } } "
            }

py_data = {17:"def getNextTag(xmlData, position):\n\tnextTag = None\n\tif xmlData!=None and xmlData!='' and position < len(xmlData) and '<' in xmlData[position:]:\n\t\twhile xmlData[position]!='<':\n\t\t\tposition+=1\n\t\tstartIndex = position\n\t\tif '>' in xmlData[position:]:\n\t\t\twhile xmlData[position] !='>':\n\t\t\t\tposition+=1\n\t\t\tnextTag = xmlData[startIndex:position+1]\n\treturn nextTag",
7:"def bottom(margin):\n\treturn pageSize.bottom(marginBottom+margin)",
12:"def toGreyScale():\n\treturn RGBColor(0.30*getRed() + 0.59*getGreen() + 0.11*getBlue())",
21:"def addClientSocketListenser(listener):\n\tif listener == None:\n\t\traise NullPointerException()\n\tlisteners.add(listener)",
23:"def addProcessToGcListLocked(proc):\n\tadded = False\n\tfor i in range(mProcessesToGc.size()-1, 0, -1):\n\t\tif mProcessesToGc.get(i).lastRequestedGc < proc.lastRequestedGc:\n\t\t\tadded = True\n\t\t\tmProcessesToGc.add(i+1, proc)\n\t\t\tbreak\n\tif not added:\n\t\tmProcessesToGc.add(0, proc)",
34:"def isFocused():\n\tif m_Control==None:\n\t\treturn False\n\treturn m_Control.isFocusControl()",
36:"def getFreeDiskSpace(checkInternal):\n\tstatus=Environment.getExternalStorageState()\n\tfreeSpace = 0\n\tif status.equals(Environment.MEDIA_MOUNTED):\n\t\tfreeSpace = freeSpaceCalculation(Environment.getExternalStorageDirectory().getPath())\n\telif checkInternal:\n\t\tfreeSpace = freeSpaceCalculation(\"/\")\n\telse:\n\t\treturn -1\n\treturn freeSpace",
38:"def removePropertyChangeListener(name, pcl):\n\tm_bcSupport.removePropertyChangeListener(name, pcl)",
40:"def incDfsUsed(value):\n\tused.addAndGet(value)",
49:"def removeElements(elements):\n\tif len(elements)>0:\n\t\tfElements.removeAll(elements)\n\t\tif fTable!=None:\n\t\t\tfTable.remove(elements.toArray())\n\t\tdialogFieldChange()"
         }

In [62]:
i=17
token = cv.getToken(py_data[i])
api = cv.getAPISequence(py_data[i])
meth = cv.getMethodName(py_data[i])

In [63]:
print(token)
print(api)
print(meth)

['len', 'next', 'tag', 'position', 'start', 'index', 'xml', 'data']
['xmlData']
['getNextTag']


In [64]:
py_token_vec = c2v.convert_tokens(token)
py_apiseq_vec = c2v.convert_apiseq(api)
py_meth_vec= c2v.convert_methname(meth)
print(py_token_vec)
print(py_apiseq_vec)
print(py_meth_vec)

[150, 21, 130, 175, 46, 19, 84, 42]
[0]
[0]


In [65]:
java_token_vec = t[i-1]
java_apiseq_vec = a[i-1]
java_meth_vec = m[i-1]
print(java_token_vec)
print(java_apiseq_vec)
print(java_meth_vec)

[ 21.  71.  55.  42.  84.  10.  46.  19. 130. 175.  56.  79.]
[ 3. 80.  3. 23.  3. 33.  3. 55.  3. 33.  3. 55.  3. 33.]
[  2.  25. 168.]


In [66]:
py_token_vec = move(py_token_vec, java_token_vec)
py_apiseq_vec = move(py_apiseq_vec, java_apiseq_vec)
py_meth_vec = move(py_meth_vec, java_meth_vec)
print(py_token_vec)
print(py_apiseq_vec)
print(py_meth_vec)

[21, 0, 0, 42, 84, 0, 46, 19, 130, 175, 0, 0, 150]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0]


In [81]:
Xs, Xt = get_vec(java_data, py_data)
labels = []
for i in range(len(Xs)):
    labels.append(i)
for i in range(len(Xs)):
    labels.append(i)
data = np.concatenate((Xs, Xt))

In [48]:
cv.printAST(pydata[i])

Module(body=[<_ast.FunctionDef object at 0x00000196C686FD30>])
  FunctionDef(name='removeElements', args=arguments(args=[<_ast.arg object at 0x00000196C686FC50>], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[<_ast.If object at 0x00000196C686F908>], decorator_list=[], returns=None)
    arguments(args=[<_ast.arg object at 0x00000196C686FC50>], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[])
      arg(arg='elements', annotation=None)
    If(test=Compare(ops=[<_ast.Gt object at 0x00000196C28F4B00>], comparators=[<_ast.Num object at 0x00000196C686FC18>]), body=[<_ast.Expr object at 0x00000196C686F550>, <_ast.If object at 0x00000196C686F940>, <_ast.Expr object at 0x00000196CCC2B1D0>], orelse=[])
      Compare(ops=[<_ast.Gt object at 0x00000196C28F4B00>], comparators=[<_ast.Num object at 0x00000196C686FC18>])
        Call(func=Name(id='len', ctx=Load()), args=[<_ast.Name object at 0x00000196C686F710>], keywords=[])
          Name(id='len', c

In [14]:
a={1:2,2:3,3:4}
a.get(1)

2

In [30]:
it = a.iteritems()

AttributeError: 'dict' object has no attribute 'iteritems'

In [29]:
next(it)

StopIteration: 